# ETL LegalRES

* Extract: Bajar los JSON de legalRES de S3 
* Transform: Seleccionar las columnas que interesan, aplanar a una tabla y generar columnas nuevas
* Load: Cargar tabla limpia a S3

In [7]:
#!pip install awswrangler
#!pip install --upgrade pandas

In [2]:
import pandas as pd
from sagemaker import get_execution_role
import boto3
from tqdm import tqdm

import awswrangler as wr

import json
from pandas.io.json import json_normalize

#data = pd.read_json(data_location)

In [3]:
bucket='legalres-storage-study-json'
data_key = 'AC000lvffws1.json'
data_location = 's3://{}/{}'.format(bucket, data_key)


s3 = boto3.client('s3')


Config datos ya extraidos para no duplicar

In [4]:
prefix_processed = 'processed'
data_location_processed = 's3://{}/{}/'.format(bucket, prefix_processed)

## Extract 

Datos ya extraidos para no duplicar

In [12]:
df_processed = wr.s3.read_parquet(path=data_location_processed,
                                  columns=['cve'])

In [17]:
df_processed.shape

(537765, 1)

In [13]:
df_processed.head()

,cve
0,AC1DrSkE1pdl
1,AC1Dv0J0V4Zr
2,AC1G6RzRn16K
3,AC1G8rKdG7Xu
4,AC1H6Pt61S4o


In [18]:
cve_processed = list(df_processed['cve'].values)

In [19]:
assert len(cve_processed) == len(df_processed.index)

In [35]:
'AC0AEHf1alMr' in cve_processed

False

Prueba con un archivo para ver estructura

In [ ]:
#data_stream = s3.get_object(Bucket=bucket,Key=data_key)['Body'].read().decode('utf-8')
#data = json.loads(data_stream)
#print(json.dumps(data, indent=4, ensure_ascii=False))

Helper functions

In [20]:
def find_item(struct, field='tipo', string=''):
    for item in struct:
        if item[field] == string:
            return item
    return None

In [21]:
def get_data_constitucion(dj):
    """
    Parses the relevant data from a JSON
    
    inputs
    ------
    
    dj: json 
        data jason
        
    
    outputs
    -------
    
    dict
        With the relevant columns
    """
    
    parsed = dict()
    
    parsed['rut'] = dj['rut']
    parsed['cve'] = dj['cve']
    parsed['fecha'] = dj['fecha']
    parsed['lugar'] = find_item(dj['estructura'][1]['articulos'],
                                 field='articulo',
                                 string='inicio')['texto']
    parsed['tipo_sociedad'] = dj['tipo_sociedad']
    
    parsed['objeto'] = find_item(dj['estructura'][1]['articulos'],
                                 field='subtitulo',
                                 string='OBJETO')['texto']
    
    parsed['firmas'] = find_item(dj['estructura'], 
                                 field='tipo',
                                 string='firmas')['firmas']
    
    return parsed
    

Objetos para recorrer carpeta con JSON

In [22]:
paginator = s3.get_paginator("list_objects_v2")
pages = paginator.paginate(Bucket=bucket)

El loop principal que recorre todos los archivos

**Tarda 12 horas en procesar toda la carpeta**

In [ ]:
LIMIT = 10
i = 0 # pages
j = 0 # constituciones

output = []
failed_cve = []

for page in tqdm(pages, total=900):
    i = i + 1
    contents = page['Contents']
    
    keys = [obj['Key'] for obj in contents]
    keys_not_processed = [key for key in keys if key.split('.')[0] not in cve_processed]
    print('Keys to look into:', len(keys_not_processed))
    print('Sample', keys_not_processed[0])
    
    for key in keys_not_processed:
        #key = obj['Key']
        #key_cve = key.split('.')
        #if key_cve not in cve_processed:
            
        # Read data
        data_stream = s3.get_object(Bucket=bucket,Key=key)['Body'].read().decode('utf-8')
        data = json.loads(data_stream)

        try:
            if data['tipo_actuacion'] == 'CONSTITUCION':
                j = j + 1
                dictout = get_data_constitucion(data)
                output.append(dictout)
        except:
            failed_cve.append(key)

#    if i >= LIMIT:
#        break





  0%|          | 0/900 [00:00<?, ?it/s]

Keys not processed: 1000
Sample AC0AEHf1alMr.json






  0%|          | 1/900 [00:54<13:41:53, 54.85s/it]

Keys not processed: 1000
Sample AC0DtAnHnwBE.json






  0%|          | 2/900 [01:53<13:59:33, 56.10s/it]

...





998it [7:39:44, 52.55s/it]

Keys not processed: 1000
Sample ACzwQfT3BkIL.json






999it [7:40:37, 52.66s/it]

Keys not processed: 133
Sample SA-76986215-3.json


UnicodeDecodeError: 'utf-8' codec can't decode byte 0xbc in position 7: invalid start byte

In [37]:
df = pd.DataFrame(output)
df.head()

,rut,cve,fecha,lugar,tipo_sociedad,objeto,firmas
0,76826743-K,AC0AEHf1alMr,2018-01-24,"En EST CENTRAL, Región METROPOLITANA DE SANTIA...",SOCIEDAD POR ACCIONES,El objeto de la sociedad será ELABORACIÓN DE O...,"[{'rut': '25.847.787-1', 'nombre': 'RIGOBERTO ..."
1,76334572-6,AC0AEVJh1zZH,2013-12-27,"En MOLINA, Región del MAULE, Chile, a 27 de di...",EMPRESA INDIVIDUAL DE RESPONSABILIDAD LIMITADA,La Empresa tendrá por objeto desarrollar las s...,"[{'rut': '15.127.386-6', 'nombre': 'JUAN ESTEB..."
2,76465246-0,AC0AEnWLwZEc,2014-12-22,"En COPIAPO, Región de ATACAMA, Chile, a 22 de ...",SOCIEDAD DE RESPONSABILIDAD LIMITADA,La Sociedad tendrá por objeto desarrollar las ...,"[{'rut': '11.422.085-K', 'nombre': 'HÉCTOR ABR..."
3,76601116-0,AC0AEncvHmOl,2016-03-16,"En LA FLORIDA, Región METROPOLITANA DE SANTIAG...",SOCIEDAD DE RESPONSABILIDAD LIMITADA,La Sociedad tendrá por objeto desarrollar las ...,"[{'rut': '18.060.612-2', 'nombre': 'VALENTINA ..."
4,76612411-9,AC0AFALYwLoJ,2016-04-20,"En PEÑALOLEN, Región METROPOLITANA DE SANTIAGO...",SOCIEDAD DE RESPONSABILIDAD LIMITADA,La Sociedad tendrá por objeto desarrollar las ...,"[{'rut': '14.667.766-5', 'nombre': 'MIRIAN KAT..."


In [38]:
df.shape

(260295, 7)

In [39]:
df.dtypes

rut              object
cve              object
fecha            object
lugar            object
tipo_sociedad    object
objeto           object
firmas           object
dtype: object

## Transform 

Campo fecha

In [40]:
df['fecha'] = pd.to_datetime(df['fecha'], format="%Y-%m-%d")

In [44]:
df['year'] = df['fecha'].dt.year
df['month'] = df['fecha'].dt.month

In [45]:
df.head()

,rut,cve,fecha,lugar,tipo_sociedad,objeto,firmas,year,month
0,76826743-K,AC0AEHf1alMr,2018-01-24,"En EST CENTRAL, Región METROPOLITANA DE SANTIA...",SOCIEDAD POR ACCIONES,El objeto de la sociedad será ELABORACIÓN DE O...,"[{'rut': '25.847.787-1', 'nombre': 'RIGOBERTO ...",2018,1
1,76334572-6,AC0AEVJh1zZH,2013-12-27,"En MOLINA, Región del MAULE, Chile, a 27 de di...",EMPRESA INDIVIDUAL DE RESPONSABILIDAD LIMITADA,La Empresa tendrá por objeto desarrollar las s...,"[{'rut': '15.127.386-6', 'nombre': 'JUAN ESTEB...",2013,12
2,76465246-0,AC0AEnWLwZEc,2014-12-22,"En COPIAPO, Región de ATACAMA, Chile, a 22 de ...",SOCIEDAD DE RESPONSABILIDAD LIMITADA,La Sociedad tendrá por objeto desarrollar las ...,"[{'rut': '11.422.085-K', 'nombre': 'HÉCTOR ABR...",2014,12
3,76601116-0,AC0AEncvHmOl,2016-03-16,"En LA FLORIDA, Región METROPOLITANA DE SANTIAG...",SOCIEDAD DE RESPONSABILIDAD LIMITADA,La Sociedad tendrá por objeto desarrollar las ...,"[{'rut': '18.060.612-2', 'nombre': 'VALENTINA ...",2016,3
4,76612411-9,AC0AFALYwLoJ,2016-04-20,"En PEÑALOLEN, Región METROPOLITANA DE SANTIAGO...",SOCIEDAD DE RESPONSABILIDAD LIMITADA,La Sociedad tendrá por objeto desarrollar las ...,"[{'rut': '14.667.766-5', 'nombre': 'MIRIAN KAT...",2016,4


## Load 

En formato Apache Parquet

In [46]:


wr.s3.to_parquet(
    df,
    path='s3://{}/processed'.format(bucket),
    dataset=True,
    partition_cols=['year', 'month'],
    mode='append'
)

{'paths': ['s3://legalres-storage-study-json/processed/year=2013/month=5/ba36a5361b4d42959871f86937ba7ab8.snappy.parquet',
  's3://legalres-storage-study-json/processed/year=2013/month=6/ba36a5361b4d42959871f86937ba7ab8.snappy.parquet',
  's3://legalres-storage-study-json/processed/year=2013/month=7/ba36a5361b4d42959871f86937ba7ab8.snappy.parquet',
  's3://legalres-storage-study-json/processed/year=2013/month=8/ba36a5361b4d42959871f86937ba7ab8.snappy.parquet',
  's3://legalres-storage-study-json/processed/year=2013/month=9/ba36a5361b4d42959871f86937ba7ab8.snappy.parquet',
  's3://legalres-storage-study-json/processed/year=2013/month=10/ba36a5361b4d42959871f86937ba7ab8.snappy.parquet',
  's3://legalres-storage-study-json/processed/year=2013/month=11/ba36a5361b4d42959871f86937ba7ab8.snappy.parquet',
  's3://legalres-storage-study-json/processed/year=2013/month=12/ba36a5361b4d42959871f86937ba7ab8.snappy.parquet',
  's3://legalres-storage-study-json/processed/year=2014/month=1/ba36a5361b4d